<a href="https://colab.research.google.com/github/zoe-weinstein/ENVIR-458/blob/main/Homework4_Weinstein.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Homework 4

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

from scipy.interpolate import interp1d
from sklearn.linear_model import LinearRegression
from scipy.optimize import minimize
from sklearn.metrics import r2_score

## 1. Data Preprocessing (1 points)
* Load the dataset using pandas and display the first few rows.
* Rename columns as specified above.
  * TA_F → TA
  * SW_IN_F → SW
  * VPD_F → VPD
  * GPP_NT_VUT_REF → GPP (Target variable)

* Drop the TIMESTAMP column as it is not needed for regression.
* Check for missing values and handle them if any exist.

In [2]:
df = pd.read_csv('Wcr_GPPdaily.csv')
df.head()

,TIMESTAMP,TA_F,SW_IN_F,VPD_F,GPP_NT_VUT_REF
0,19990101,-20.063,72.603,0.413,-0.517364
1,19990102,-12.814,12.358,0.147,-0.094241
2,19990103,-12.625,33.132,0.128,-0.166819
3,19990104,-18.652,93.481,0.263,-0.582301
4,19990105,-20.269,45.502,0.261,-0.568240


In [3]:
df = df.rename(columns= {"TA_F": "TA", "SW_IN_F": "SW", "VPD_F": "VPD", "GPP_NT_VUT_REF": "GPP"})
df.head()

,TIMESTAMP,TA,SW,VPD,GPP
0,19990101,-20.063,72.603,0.413,-0.517364
1,19990102,-12.814,12.358,0.147,-0.094241
2,19990103,-12.625,33.132,0.128,-0.166819
3,19990104,-18.652,93.481,0.263,-0.582301
4,19990105,-20.269,45.502,0.261,-0.568240


In [4]:
df.isnull().sum()

,0
TIMESTAMP,0
TA,0
SW,0
VPD,0
GPP,0


### 2. Feature Engineering - Interaction Terms (1 points)
Create interaction terms between the predictors:
SW×VPD,SW×TA,VPD×TA
Add these interaction terms as new columns in the dataset.

In [5]:
df["SW_VPD"] = df["SW"] * df["VPD"]
df["SW_TA"] = df["SW"] * df["TA"]
df["VPD_TA"] = df["VPD"] * df["TA"]
df.head()

,TIMESTAMP,TA,SW,VPD,GPP,SW_VPD,SW_TA,VPD_TA
0,19990101,-20.063,72.603,0.413,-0.517364,29.985039,-1456.633989,-8.286019
1,19990102,-12.814,12.358,0.147,-0.094241,1.816626,-158.355412,-1.883658
2,19990103,-12.625,33.132,0.128,-0.166819,4.240896,-418.291500,-1.616000
3,19990104,-18.652,93.481,0.263,-0.582301,24.585503,-1743.607612,-4.905476
4,19990105,-20.269,45.502,0.261,-0.568240,11.876022,-922.280038,-5.290209


## 3. Build a Linear Regression Model using SciPy Optimization (3 points)
* Define the linear model equation:
GPP=β0​+β1​⋅SW+β2​⋅VPD+β3​⋅TA+β4​⋅(SW×VPD)+β5​⋅(SW×TA)+β6​⋅(VPD×TA)
* Use SciPy's minimize function to find the best-fit parameters (β0​,β1​,β2​,β3​,β4​,β5​,β6​) by minimizing the Mean Squared Error (MSE).
* Print the optimized coefficients.

In [6]:
# Extract variables from DataFrame
SW = df["SW"].values
VPD = df["VPD"].values
TA = df["TA"].values
GPP = df["GPP"].values
SW_VPD = df["SW_VPD"].values
SW_TA = df["SW_TA"].values
VPD_TA = df["VPD_TA"].values

# Define the linear model equation
def linear_model(beta, SW, VPD, TA, SW_VPD, SW_TA, VPD_TA):
    return (
        beta[0] + beta[1] * SW + beta[2] * VPD + beta[3] * TA +
        beta[4] * SW_VPD + beta[5] * SW_TA + beta[6] * VPD_TA
    )

# Define the cost function (Mean Squared Error)
def mse_loss(beta, SW, VPD, TA, SW_VPD, SW_TA, VPD_TA, GPP):
    predictions = linear_model(beta, SW, VPD, TA, SW_VPD, SW_TA, VPD_TA)
    return np.mean((GPP - predictions) ** 2)

# Initial guess for parameters (β0 to β6)
beta_initial = np.zeros(7)

# Perform optimization to minimize MSE
result = minimize(mse_loss, beta_initial, args=(SW, VPD, TA, SW_VPD, SW_TA, VPD_TA, GPP), method="BFGS")

# Print the optimized coefficients
optimized_beta = result.x
print("Optimized Coefficients:")
for i, coef in enumerate(optimized_beta):
    print(f"β{i}: {coef}")

Optimized Coefficients:
β0: -944.6391385761914
β1: -3.245829012213404
β2: -648.3947259737711
β3: 82.15079008795271
β4: 1.7916242530440003
β5: -0.45600827057720106
β6: 13.597198669184277


## 4. Build a Linear Regression Model using sklearn's Linear Regression Function (3 points)

In [7]:
# Build Linear Regression Model using sklearn
X = np.column_stack((SW, VPD, TA, SW_VPD, SW_TA, VPD_TA))
reg = LinearRegression().fit(X, GPP)

# Extract and print coefficients
print("\nCoefficients from sklearn's Linear Regression:")
print(f"Intercept: {reg.intercept_}")
for i, coef in enumerate(reg.coef_):
    print(f"β{i+1}: {coef}")


Coefficients from sklearn's Linear Regression:
Intercept: -944.0672018204632
β1: -3.2491496798124846
β2: -648.5741732974528
β3: 82.16998132929432
β4: 1.792491510791507
β5: -0.45605035694927665
β6: 13.595402450063464


## 5. Compare the Models (2 points)

In [8]:
# Compute R-squared for both models
GPP_pred_scipy = linear_model(optimized_beta, SW, VPD, TA, SW_VPD, SW_TA, VPD_TA)
GPP_pred_sklearn = reg.predict(X)

r2_scipy = r2_score(GPP, GPP_pred_scipy)
r2_sklearn = r2_score(GPP, GPP_pred_sklearn)

print("\nR-squared Comparison:")
print(f"SciPy Optimization Model R²: {r2_scipy}")
print(f"sklearn Linear Regression R²: {r2_sklearn}")



R-squared Comparison:
SciPy Optimization Model R²: 0.01358269810244761
sklearn Linear Regression R²: 0.013582700933255087
